# LGBM - Training

Sources
1. [LGBM & Deberta Explained by ZULQARNAIN ALI](https://www.kaggle.com/code/zulqarnainalipk/lgbm-deberta-explained)

## Setup

In [1]:
import os
import pandas as pd
import numpy as np
from sklearn.metrics import cohen_kappa_score
import lightgbm as lgb
import pickle as pkl
import wandb
from lightgbm import log_evaluation, early_stopping
from sklearn.metrics import cohen_kappa_score
from sklearn.model_selection import StratifiedKFold

In [2]:
os.chdir("../../")

In [3]:
from lib.config import config
from lib.paths import Paths
from lib.utils.utils import seed_everything
from lib.criterion.metrics import log_metrics
from lib.model.utils import quadratic_weighted_kappa, qwk_obj, get_score

In [4]:
seed_everything()

### Setting Up WandB

In [5]:
WANDB_PROJECT = "Kaggle_ASE_2.0"
WANDB_NAME = f"LGBM-ASE-12"

In [6]:
wandb.login(key=os.environ.get('WANDB_API_KEY'))
wandb.init(
    project=WANDB_PROJECT,
    name=WANDB_NAME,
    save_code=True,
    job_type="train",
    config=config,
)
config = wandb.config

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: shakleenishfar (laplacesdemon43). Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /home/ishfar/.netrc


## Model Training

### Dataset Creation

In [7]:
df = pd.read_csv(Paths.FEATURE_ENGG_CSV_PATH)
df.shape

(17307, 154)

In [8]:
ids = df.loc[:, "essay_id"]
X, y = df.drop(columns=["score", "essay_id"]), df.loc[:, "score"]
del df

X.shape, y.shape

((17307, 152), (17307,))

### Callbacks

In [9]:
callbacks = [
    log_evaluation(period=25),
    early_stopping(stopping_rounds=75, first_metric_only=True),
]

### Constants

In [10]:
a = config.lgbm_a
b = config.lgbm_b

### Training Loop

In [11]:
def train_loop(X_train, y_train, X_valid, y_valid):
    y_train = y_train.astype(np.float32) - a
    y_valid = y_valid.astype(np.float32) - a

    model = lgb.LGBMRegressor(
        objective=qwk_obj,
        metrics="None",
        learning_rate=0.01,
        max_depth=5,
        num_leaves=10,
        colsample_bytree=0.3,
        reg_alpha=0.7,
        reg_lambda=0.1,
        n_estimators=700,
        random_state=config.random_seed,
        extra_trees=True,
        class_weight="balanced",
        n_jobs=6,
        verbosity=0,
    )

    # Train model
    predictor = model.fit(
        X_train,
        y_train,
        eval_names=["train", "valid"],
        eval_set=[(X_train, y_train), (X_valid, y_valid)],
        eval_metric=quadratic_weighted_kappa,
        callbacks=callbacks,
    )

    return model, predictor

In [12]:
def validate_model(idx, X_valid, y_valid, id_valid, predictor):
    y_valid_int = y_valid.astype(np.int64).to_numpy()
    preds = predictor.predict(X_valid) + a
    preds = preds.clip(1, 6).round().astype(np.int64)

    oof = pd.DataFrame(
        {
            "essay_id": id_valid,
            "score": y_valid_int - 1,
            "pred_score": preds - 1,
        }
    )
    log_metrics(oof, f"Fold {idx}")

    return oof

### Fold Loop

In [13]:
skf = StratifiedKFold(
    n_splits=config.lgbm_n_folds,
    random_state=config.random_seed,
    shuffle=True,
)

In [14]:
oof_df = pd.DataFrame()

for idx, (train_idx, valid_idx) in enumerate(skf.split(X, y)):
    X_train, y_train = X.loc[train_idx], y.loc[train_idx]
    X_valid, y_valid = X.loc[valid_idx], y.loc[valid_idx]
    id_valid = ids.loc[valid_idx].to_numpy()

    model, predictor = train_loop(X_train, y_train, X_valid, y_valid)

    with open(f"output/LGBM/{idx}.pkl", "wb") as file:
        pkl.dump({"predictor": predictor, "model": model}, file)

    oof = validate_model(idx, X_valid, y_valid, id_valid, predictor)
    oof_df = pd.concat([oof_df, oof])

[LightGBM] [Info] Using self-defined objective function
Training until validation scores don't improve for 75 rounds
[25]	train's QWK: 0.437539	valid's QWK: 0.453591
[50]	train's QWK: 0.735501	valid's QWK: 0.749528
[75]	train's QWK: 0.798727	valid's QWK: 0.814387
[100]	train's QWK: 0.840442	valid's QWK: 0.845246
[125]	train's QWK: 0.860425	valid's QWK: 0.871054
[150]	train's QWK: 0.867924	valid's QWK: 0.873171
[175]	train's QWK: 0.872952	valid's QWK: 0.879047
[200]	train's QWK: 0.875556	valid's QWK: 0.882758
[225]	train's QWK: 0.878105	valid's QWK: 0.88927
[250]	train's QWK: 0.880296	valid's QWK: 0.893941
[275]	train's QWK: 0.882243	valid's QWK: 0.893114
[300]	train's QWK: 0.882916	valid's QWK: 0.89414
[325]	train's QWK: 0.883641	valid's QWK: 0.895287
[350]	train's QWK: 0.884381	valid's QWK: 0.895287
[375]	train's QWK: 0.884956	valid's QWK: 0.896135
[400]	train's QWK: 0.88552	valid's QWK: 0.895673
[425]	train's QWK: 0.885957	valid's QWK: 0.894936
[450]	train's QWK: 0.886288	valid's QWK

/media/ishfar/New Volume/Studies/Projects/Kaggle-Automated-Essay-Scoring/lib/criterion/metrics.py:34: RuntimeWarning: invalid value encountered in scalar divide
  precision = tp / (tp + fp)


[LightGBM] [Info] Using self-defined objective function
Training until validation scores don't improve for 75 rounds
[25]	train's QWK: 0.454453	valid's QWK: 0.426925
[50]	train's QWK: 0.730213	valid's QWK: 0.725063
[75]	train's QWK: 0.795047	valid's QWK: 0.783578
[100]	train's QWK: 0.836073	valid's QWK: 0.822033
[125]	train's QWK: 0.856877	valid's QWK: 0.849248
[150]	train's QWK: 0.867615	valid's QWK: 0.870823
[175]	train's QWK: 0.872187	valid's QWK: 0.873198
[200]	train's QWK: 0.874546	valid's QWK: 0.873563
[225]	train's QWK: 0.877661	valid's QWK: 0.875794
[250]	train's QWK: 0.879918	valid's QWK: 0.875495
[275]	train's QWK: 0.881429	valid's QWK: 0.877567
[300]	train's QWK: 0.883009	valid's QWK: 0.880586
[325]	train's QWK: 0.884046	valid's QWK: 0.88032
[350]	train's QWK: 0.884637	valid's QWK: 0.883357
[375]	train's QWK: 0.884836	valid's QWK: 0.883532
[400]	train's QWK: 0.885102	valid's QWK: 0.883846
[425]	train's QWK: 0.885462	valid's QWK: 0.884041
Early stopping, best iteration is:
[3

/media/ishfar/New Volume/Studies/Projects/Kaggle-Automated-Essay-Scoring/lib/criterion/metrics.py:34: RuntimeWarning: invalid value encountered in scalar divide
  precision = tp / (tp + fp)


[LightGBM] [Info] Using self-defined objective function
Training until validation scores don't improve for 75 rounds
[25]	train's QWK: 0.450224	valid's QWK: 0.45675
[50]	train's QWK: 0.731138	valid's QWK: 0.693839
[75]	train's QWK: 0.798021	valid's QWK: 0.775816
[100]	train's QWK: 0.83821	valid's QWK: 0.820967
[125]	train's QWK: 0.859641	valid's QWK: 0.847212
[150]	train's QWK: 0.869508	valid's QWK: 0.851537
[175]	train's QWK: 0.873822	valid's QWK: 0.858553
[200]	train's QWK: 0.877095	valid's QWK: 0.857365
[225]	train's QWK: 0.880035	valid's QWK: 0.857747
[250]	train's QWK: 0.882045	valid's QWK: 0.863606
[275]	train's QWK: 0.883428	valid's QWK: 0.865765
[300]	train's QWK: 0.884728	valid's QWK: 0.865146
[325]	train's QWK: 0.885343	valid's QWK: 0.8651
[350]	train's QWK: 0.886313	valid's QWK: 0.865035
Early stopping, best iteration is:
[277]	train's QWK: 0.883464	valid's QWK: 0.866516
Evaluated only: QWK
[LightGBM] [Info] Using self-defined objective function
Training until validation sco

/media/ishfar/New Volume/Studies/Projects/Kaggle-Automated-Essay-Scoring/lib/criterion/metrics.py:34: RuntimeWarning: invalid value encountered in scalar divide
  precision = tp / (tp + fp)


[LightGBM] [Info] Using self-defined objective function
Training until validation scores don't improve for 75 rounds
[25]	train's QWK: 0.455764	valid's QWK: 0.472955
[50]	train's QWK: 0.733186	valid's QWK: 0.716572
[75]	train's QWK: 0.799064	valid's QWK: 0.791741
[100]	train's QWK: 0.834982	valid's QWK: 0.827631
[125]	train's QWK: 0.858411	valid's QWK: 0.848697
[150]	train's QWK: 0.868377	valid's QWK: 0.858521
[175]	train's QWK: 0.873489	valid's QWK: 0.865231
[200]	train's QWK: 0.877055	valid's QWK: 0.865095
[225]	train's QWK: 0.879759	valid's QWK: 0.867765
[250]	train's QWK: 0.881473	valid's QWK: 0.872468
[275]	train's QWK: 0.88304	valid's QWK: 0.874755
[300]	train's QWK: 0.88453	valid's QWK: 0.875688
[325]	train's QWK: 0.885085	valid's QWK: 0.879686
[350]	train's QWK: 0.885509	valid's QWK: 0.880381
[375]	train's QWK: 0.886258	valid's QWK: 0.882208
[400]	train's QWK: 0.886177	valid's QWK: 0.882208
Early stopping, best iteration is:
[338]	train's QWK: 0.88515	valid's QWK: 0.882641
Eval

In [15]:
score = get_score(oof_df["score"], oof_df["pred_score"])
print(f'Overall Score: {score:<.4f}')
wandb.log({"CV/qwk_score": score})

Overall Score: 0.8843


In [16]:
wandb.log({"oof_table": wandb.Table(dataframe=oof_df)})

In [17]:
log_metrics(
    pd.DataFrame(
        {
            "score": oof_df.score.to_numpy(),
            "pred_score": oof_df.pred_score.to_numpy(),
        }
    ),
    "Overall",
)

In [18]:
wandb.finish()

CV/qwk_score,▁
CV/qwk_score,0.88428
